In [ ]:
# Standard libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from ipywidgets import IntProgress
from IPython.display import display

# Append base directory
import os,sys,inspect
rootname = "pub-2020-exploratory-analysis"
thispath = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
rootpath = os.path.join(thispath[:thispath.index(rootname)], rootname)
sys.path.append(rootpath)
print("Appended root directory", rootpath)

from mesostat.utils.qt_helper import gui_fnames, gui_fpath
from mesostat.metric.metric import MetricCalculator
from mesostat.utils.hdf5_io import DataStorage

from lib.sych.data_fc_db import DataFCDatabase
from lib.sych.metric_helper import metric_by_session
from lib.sych.plot_helper import imshow_dataset_by_mouse

%load_ext autoreload
%autoreload 2

In [ ]:
# tmp_path = root_path_data if 'root_path_data' in locals() else "./"
params = {}
# params['root_path_data']  = gui_fpath("Path to data collection",  './')
params['root_path_data'] = '/media/alyosha/Data/TE_data/yarodata/neuro'

#params['root_path_paw']   = gui_fpath("Path to Paw data",         params['root_path_data'])
#params['root_path_lick']  = gui_fpath("Path to Lick data",        params['root_path_paw'])
#params['root_path_whisk'] = gui_fpath("Path to Whisk Data",       params['root_path_lick'])
# params['root_path_te']    = gui_fpath("Select TE results folder", params['root_path_data'])

In [ ]:
dataDB = DataFCDatabase(params)

In [ ]:
dataDB.read_te_files()
dataDB.read_neuro_files()
dataDB.read_resample_paw_files()
dataDB.read_resample_lick_files()   # Currently does not work. Ask Yaro if we need it
dataDB.read_resample_whisk_files()

In [ ]:
pTHR = 0.7
dataDB.mark_days_expert_naive(pTHR)

# Multi-Regional Metrics

**Main Messages**:
* System time-specificity, how it changes with sessions?
* Do bulk metrics deliver more info than mean?
* How metrics depend on preprocessing?

**Measures of instantainous redundancy**
- Mean (for comparison)
- Average Correlation
- PCA(1) - ?
- Entropy, TC

**Measures of temporal relations**
- PI_ND

**Plot strategies**
- 2D-Session-vs-timestep
- 1D-Session/Pefrormance for (texture-avg-time/reward-avg-time)
    - Have significant version with test of phase vs remainder of trial
- all plots compare preprocessing
    
## TODO
1. Debug
    * Entropy jumps between sessions: Check if entropy outliers correlate with number of trials
    * Test NPEET metrics for convergence.
    * Check why TC is sometimes negative, it should not be

In [ ]:
mc = MetricCalculator(serial=False, verbose=False)

In [ ]:
ds = DataStorage('sych_result_population_bulk_info.h5')

## 1.1 Average Correlation

In [ ]:
window = 6
for iMouse, mousename in enumerate(sorted(dataDB.mice)):
    metric_by_session(dataDB, mc, ds, mousename, 'avgcorr', 's',
                      dataName='avgcorr_iGO',
                      trialType='iGO',
                      cropTime=200-window+1,
                      timeWindow=window)

## 1.2 Average Entropy

In [ ]:
window = 6
for iMouse, mousename in enumerate(sorted(dataDB.mice)):
    metric_by_session(dataDB, mc, ds, mousename, 'avg_entropy', 's',
                      dataName='avg_entropy_iGO',
                      trialType='iGO',
                      cropTime=200-window+1,
                      timeWindow=window)

## 1.3 Average Total Correlation

In [ ]:
window = 6
for iMouse, mousename in enumerate(sorted(dataDB.mice)):
    metric_by_session(dataDB, mc, ds, mousename, 'avg_TC', 's',
                      dataName='avg_TC_iGO',
                      trialType='iGO',
                      cropTime=200-window+1,
                      timeWindow=window)

## 1.4 Average Predictive Information

In [ ]:
window = 6
for iMouse, mousename in enumerate(sorted(dataDB.mice)):
    metric_by_session(dataDB, mc, ds, mousename, 'avg_PI', 's',
                      dataName='avg_PI_iGO',
                      trialType='iGO',
                      cropTime=200-window+1,
                      timeWindow=window,
                      metricSettings={"max_lag" : 1})

In [ ]:
imshow_dataset_by_mouse(dataDB, ds, 'avgcorr_iGO', limits=[0,1], fig1size=(10,3))

In [ ]:
imshow_dataset_by_mouse(dataDB, ds, 'avg_entropy_iGO')

In [ ]:
imshow_dataset_by_mouse(dataDB, ds, 'avg_TC_iGO')

In [ ]:
imshow_dataset_by_mouse(dataDB, ds, 'avg_PI_iGO')